In [ ]:
# Importing libraries
import geopy
import geopy.distance
import time
import googlemaps 
import pandas as pd
import folium
import datetime

# API Key input
API_KEY = 'Your API Key here'

# Creating lists
bearings = [0, 45, 90, 135, 180, 225, 270, 315]
business_list = []
distance = 2000
lat = -30.015657 
lon = -51.188767


# Start point
start = geopy.Point(lat, lon)
d = geopy.distance.distance(meters=distance/2)

# Creates multiple lat/lon coordinates within the initial radius and extracts data from all of them
coordinates = [start]

for i in bearings:
    final = d.destination(point=start, bearing=i)
    coordinates.append(final)

for i in range(len(coordinates)):    
    lat_long=str(coordinates[i][0:2]).replace('(','').replace(')','')
        
    map_client = googlemaps.Client(API_KEY)
    search_string = 'school'        
    response = map_client.places_nearby(
        location=(lat_long),
        keyword=search_string,
        radius=distance,
        open_now=False
    )   

    business_list.extend(response.get('results'))
    next_page_token = response.get('next_page_token')

    while next_page_token:
        time.sleep(2)
        response = map_client.places_nearby(
            location=(lat_long),
            keyword=search_string,
            radius=distance,
            open_now=False,
            page_token=next_page_token
        )   
        business_list.extend(response.get('results'))
        next_page_token = response.get('next_page_token')

df = pd.DataFrame(business_list)
df['url'] = 'https://www.google.com/maps/place/?q=place_id:' + df['place_id']

# Droping columns
columns_to_drop = ['geometry',
                   'icon', 
                   'icon_background_color',
                   'icon_mask_base_uri',
                   'place_id',
                   'plus_code',
                   'reference',
                   'scope',
                   'opening_hours',
                   'user_ratings_total',
                   'rating',
                   'photos']

for column in columns_to_drop:
    del df[column]

df.drop_duplicates(subset='name', keep='first', inplace=True)    

# Creating .xlsx file 
x = datetime.datetime.now()
date = x.strftime("%d%b%y-%H%M%S")
name = 'APIEscolas' + date + '.xlsx'

df.to_excel(name)

print(name)

# Ploting map showing the selected area 
map = folium.Map(location=[lat, lon], zoom_start=12)

folium.Circle([lat, lon],
               radius=distance*1.5,
               fill=False,
               opacity=0.5,
               ).add_to(map)
 
map